In [144]:
import numpy as np
import pandas as pd
import math
from math import log
import copy

In [133]:
class DecisionTree:
    def __init__(self):
        self.mytree=None
    
    def  cal_entropy(self,datasets):#计算某一列的信息熵
        num1=len(datasets)
        labelcount={}
        for num in datasets:
            labelcount1=datasets.count(num)#统计各个标签出现的次数
            labelcount[num]=labelcount1
        entropy=-sum([(entropy1/num1)*log(entropy1/num1,2) for entropy1 in labelcount.values()])
        return entropy 
        
    def condational_entropy(self,datasets,axis=0):#先指定计算某一列的经验条件熵
        num1=len(datasets)
        featuresaxis=[]
        for i in range(num1):
            featurescate=datasets[i][axis]#把数据集中的单独一列挑选出来，并放入featureaxis,此函数默认为第一列
            featuresaxis.append(featurescate)
        uniquefeatures=list(set(featuresaxis))#看单独一列中有那些属性值
        cond_entropy=0
        for feauture in uniquefeatures:#分别筛选特征值
            data=[]
            for k in range(num1):
                if datasets[k][axis]==feauture:
                    data.append(datasets[k][-1])
                    cal=self.cal_entropy(data)
            cond_entropy+=(len(data)/num1)*cal
        return cond_entropy
    
    def max_gain(self,datasets):#计算每一列的信息增益，并挑选出信息增益最大的列所对应的索引值
        feauturecount=len(datasets[0])
        num1genjiedian=len(datasets)
        genjiedian=[]#单独拿出根节点，并计算根节点的信息熵
        for c in range(num1genjiedian):
            genjiedian.append(datasets[c][-1])
        ent=self.cal_entropy(genjiedian)
        gaininfo=[]
        for m in range(feauturecount-1):
            cond_entropy=self.condational_entropy(datasets,m)
            gain=ent-cond_entropy#计算每一个属性的信息增益
            gaininfo.append(gain)
        maxindex=gaininfo.index(max(gaininfo))
        #print('信息增益最大的节点是：'+str(maxindex)+'，值为：'+str(max(gaininfo)))
        return maxindex
    
    def splitData(self,trainData,index,value):#计算子集，即后续划分的数据集，目的是返回最佳属性对应的特征值，将不同的特征值筛选出来
        subData = []
        for trainLine in trainData:
            if trainLine[index]==value:#这一行会将第index+1列与values相等的所有行均选出来
                reducedFeatVec = []
                for i in range(len(trainLine)):#循环将所有非index+1列的行数据添加到列表中
                    if i==index:#跳过第index+1列
                        continue
                    reducedFeatVec.append(trainLine[i])
                subData.append(reducedFeatVec)
        return subData

    def fit(self,traindata):#树是一个多重嵌套的字典结构
        classlist=[temp[-1] for temp in traindata]
        classset=set(classlist)#看根节点有多数个不同的标签，如果所有标签都是同一个值，则直接将其作为决策树节点，并结束决策树划分
        if len(classset)==1:
            return classlist[0]
        if len(traindata[0])==1:#如果特征集为空，只有标签集，即只有D存在，则将D中最大的类Dk作为标签
            mk={}
            for c in traindata[0]:
                mk1=traindata[0].count(c)
                mk[c]=mk1
            return max(mk,key=mk.get)
        best_feature=self.max_gain(traindata)#计算当前数据集的最佳属性对应的索引
        mytree={best_feature:{}}
        featurevalues=[example[best_feature] for example in traindata]
        uniquefeature=list(set(featurevalues))
        for values in uniquefeature:
            mytree[best_feature][values]=self.fit(self.splitData(traindata,best_feature,values))#递归实现树结构
        self.mytree=mytree
        return mytree
            
    def classify(self,testData,dTrees):#这里假设testdata为一个一维的数组
        index = list(dTrees.keys())[0]
        secondDict = dTrees[index]#该属性下对应的树结构
        testValue = testData[index]#测试集相应属性的值
        for key in secondDict.keys():
            if testValue==key:#按照属性相应的值进行查找
                if type(secondDict[key])==dict:
                    secondTest = copy.deepcopy(testData)
                    del secondTest[index]#删掉每一次递归重复的
                    classLabel = classify(secondTest,secondDict[key])#递归查找结果
                else:
                    classLabel = secondDict[key]
        return classLabel
    
    def predict(self,testdata):#testdata为一个具有多个测试样本的二维数组
        classlabel1=[]
        for testline in testdata:
            classlabel=self.classify(testline,self.mytree)
            classlabel1.append(classlabel)
        return classlabel1

In [97]:
 datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]

In [148]:
dt=DecisionTree()
best=dt.max_gain(datasets)
dt1=dt.fit(datasets)
dt.predict([['老年', '否', '否', '一般']])

['否']